# Quickstart : compare runs, choose a model, and deploy it to a REST API

- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLFlow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

In [27]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe # Pour l'optimisation des hyperparamètres
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [6]:
# Load dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
                sep =";"

)

In [7]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [8]:
# Split the data into training, validation and test sets

train, test = train_test_split(data, test_size = 0.25, random_state = 42)


In [9]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [42]:
train_x = train.drop(['quality'], axis = 1).values
train_y = train[['quality']].values.ravel()

# Splitting this train data into train and validation
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.20, random_state = 42)

# Test dataset
test_x = test.drop(['quality'], axis = 1).values
test_y = test[['quality']].values.ravel()

signature = infer_signature(train_x, train_y)



In [43]:
keras.Sequential()

<Sequential name=sequential_1, built=False>

In [55]:
# ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y) :
    # Define the model architecture
    mean = np.mean(train_x, axis = 0)
    var = np.var(train_x, axis = 0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean = mean, variance = var),
            keras.layers.Dense(64, activation = "relu"),
            keras.layers.Dense(1)
        ]
    )

    # Compile
    model.compile(optimizer = keras.optimizers.SGD(
        learning_rate = params["lr"], momentum = params["momentum"]
    ),
    loss = "mean_squared_error",
    metrics = [keras.metrics.RootMeanSquaredError()]
    )

    # Train the ANN Model with lr and momentum params with mlflow tracking
    with mlflow.start_run(nested = True) :
        model.fit(train_x, train_y, validation_data = (valid_x, valid_y),
                  epochs = epochs,
                  batch_size = 64)
        
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size = 64)

        eval_rmse = eval_result[1]

        # Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model 
        mlflow.tensorflow.log_model(model, "model", signature = signature)

        return {"loss" : eval_rmse, "status" : STATUS_OK, "model" : model}


In [56]:
def objective(params) :
    # MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs = 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y
    )
    return result

In [57]:
space = {
    "lr" : hp.loguniform("lr", np.log(1e-5), np.log(1e-1)), # le lr va de 1e-5 à 1e-1 de manière log uniforme
    "momentum" : hp.uniform("momentum", 0.0, 1.0), # le moment va de 0 à 1 de manière uniforme
} # L'espace de valeurs

In [59]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run() :
    # Conduct the hyperparameter search using hyperopt
    trials = Trials()
    best = fmin(
        fn = objective, 
        space = space, 
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key = lambda x : x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature = signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters : {best}")
    print(f"Best eval rsme : {best_run['loss']}")

2024/12/18 00:33:17 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 212ms/step - loss: 29.2736 - root_mean_squared_error: 5.4105
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.4148 - root_mean_squared_error: 1.9609 - val_loss: 0.6860 - val_root_mean_squared_error: 0.8282

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6428 - root_mean_squared_error: 0.8017
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6300 - root_mean_squared_error: 0.7936 - val_loss: 0.8251 - val_root_mean_squared_error: 0.9084

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.6664 - root_mean_squared_error: 0.8163
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6413 - root_mean_squared_error: 0.8007 - val_loss: 0.5753 - val_root_mean_squared_error: 0.7585

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4739 - root_mean_squared_error: 0.6884
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [60]:
# Inferencing the model
# Code c/c du MLFlow UI
from mlflow.models import validate_serving_input

model_uri = 'runs:/7282bf75ee0e43788d8647fa08598785/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[6.218152 ],
       [6.809108 ],
       [6.325463 ],
       ...,
       [5.9369016],
       [6.5436573],
       [5.695154 ]], dtype=float32)

In [62]:
import mlflow
logged_model = 'runs:/7282bf75ee0e43788d8647fa08598785/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step


array([[6.218152 ],
       [6.809108 ],
       [6.325463 ],
       ...,
       [5.9369016],
       [6.5436573],
       [5.695154 ]], dtype=float32)

In [63]:
# Register in the model registry
# Pour l'enregistrer manuellement
mlflow.register_model(model_uri, "wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1734479087296, current_stage='None', description=None, last_updated_timestamp=1734479087296, name='wine-quality', run_id='7282bf75ee0e43788d8647fa08598785', run_link=None, source='file:///c:/Users/Colin/Desktop/MLOps/MLFlow/DLProject/mlruns/611965114172724903/7282bf75ee0e43788d8647fa08598785/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>